In [8]:
from utils import *

In [24]:
new_list = [["Item name", "Product"], ["xyz1234", "abcd2345"]]

In [25]:
df_new = pd.DataFrame(new_list[1:], columns=new_list[0])

In [26]:
df_new

,Item name,Product
0,xyz1234,abcd2345


In [18]:
def read_write_append_gs(sheet_identifier: str, mode: str, data: Union[List[List[str]], pd.DataFrame], worksheet_name: str = None, use_open: bool = True) -> pd.DataFrame:
    """
    Perform read, write, or append operations on a Google Sheets document and return the data as a Pandas DataFrame.

    Parameters:
    sheet_identifier (str): The name or ID of the Google Sheets document.
    mode (str): The mode of operation to perform, which can be 'read', 'write', or 'append'.
    data (list of lists or Pandas DataFrame): The data to write or append to the document. Should be a list of lists, where each inner list represents a row of data.
    credentials_file_path (str): The path to the Google Sheets credentials file.
    worksheet_name (str): The name of the worksheet to use. Optional; if not provided, the first worksheet in the document will be used.
    use_open (bool): Whether to use the `open` method (True) or `open_by_key` method (False) to open the document. Default is True.

    Returns:
    Pandas DataFrame: The data in the Google Sheets document, returned as a Pandas DataFrame.
    """

    # Set up the Google Sheets API credentials
    # scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    # credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file_path, scope)
    # gc = gspread.authorize(credentials)
    gc = gspread.oauth()
    # Open the Google Sheets document
    try:
        if use_open:
            sheet = gc.open(sheet_identifier)
        else:
            sheet = gc.open_by_key(sheet_identifier)
    except gspread.exceptions.SpreadsheetNotFound:
        raise ValueError(f"Spreadsheet '{sheet_identifier}' not found.")

    if worksheet_name:
        # Open the specified worksheet by name
        worksheet = sheet.worksheet(worksheet_name)
    else:
        # Use the first worksheet in the document if no name is specified
        worksheet = sheet.sheet1

    # Perform the specified mode of operation
    if mode == 'read':
        # Read the data from the document and return as a Pandas DataFrame
        data = worksheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])
        return df

    elif mode == 'write':
        # Write the data to the document and return as a Pandas DataFrame
        if isinstance(data, pd.DataFrame):
            worksheet.clear()
            data = data.astype(str)
            worksheet.update([data.columns.values.tolist()] + data.values.tolist())
            return data
        elif isinstance(data, list):
            worksheet.clear()
            worksheet.update(data)
            return pd.DataFrame(data[1:], columns=data[0])
        else:
            raise ValueError("Invalid data type for 'write' mode. Data must be a Pandas DataFrame or list of lists.")

    elif mode == 'append':
        # Append the data to the document and return the updated data as a Pandas DataFrame
        if isinstance(data, pd.DataFrame):
            worksheet.append_rows(data.values.tolist())
            updated_data = worksheet.get_all_values()
            df = pd.DataFrame(updated_data[1:], columns=updated_data[0])
            #
            return df
        elif isinstance(data, list):
            worksheet.append_rows(data)
            updated_data = worksheet.get_all_values()
            df = pd.DataFrame(updated_data[1:], columns=updated_data[0])
            #
            return df
    # Raise an error if an invalid mode is specified
    else:
        raise ValueError("Invalid mode specified. Mode must be 'read', 'write', or 'append'.")

In [27]:
read_write_append_gs(sheet_identifier ='Alias_to_train',worksheet_name='New',mode='append',data= df_new)

,Item name,Product
0,xyz,abcd
1,xyz,abcd
2,xyz,abcd
3,xyz1234,abcd2345
